# TP4, INF8225 2025, Projet


## Imports

In [1]:
from IPython.display import clear_output

%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
%pip install numpy
%pip install swig
%pip install box2d
%pip install pygame
%pip install gymnasium
%pip install "gymnasium[box2d]"
%pip install matplotlib
%pip install wandb

clear_output()

In [ ]:
import numpy as np
import gymnasium as gym
import pygame

## Implementation

### Human Testing

In [ ]:
SEED = 42

env = gym.make("CarRacing-v3", render_mode="human", domain_randomize=False, max_episode_steps=10000, lap_complete_percent=0.95)
obs = env.reset(seed=SEED)[0]

pygame.init()
win = pygame.display.set_mode((1000, 1000))
pygame.display.set_caption("Keyboard Controller")

clock = pygame.time.Clock()

running = True
action = np.array([0.0, 0.0, 0.0])

def process_keys():
    keys = pygame.key.get_pressed()
    steer = 0.0
    gas = 0.0
    brake = 0.0

    if keys[pygame.K_LEFT]:
        steer = -1.0
    elif keys[pygame.K_RIGHT]:
        steer = 1.0

    if keys[pygame.K_UP]:
        gas = 1.0
    if keys[pygame.K_DOWN]:
        brake = 1.0

    return np.array([steer, gas, brake])

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = process_keys()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

    if done:
        obs = env.reset(seed=SEED)[0]

    clock.tick(60)

env.close()
pygame.quit()